In [2]:
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish

from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os 
import schedule

import pandas as pd
import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import style

from copy import deepcopy
from IPython.display import clear_output

In [3]:
# def realtimetotalstep():
conn = MongoClient('120.126.136.17')
db = conn.Tracker
collection = [db.rank]# db.db2, db.dn2, db.james, db.leo
for col in collection:
    cursor = col.find({})
    df = pd.DataFrame(list(cursor))
    
# df

In [66]:
def insert_rank_relationship():
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker
    collection = [db.prediction]# db.db2, db.dn2, db.james, db.leo
    server = "120.126.136.17"
    port = 1883 #port
    
    for col in collection:
        cursor = col.find({})
        prediction = pd.DataFrame(list(cursor))
    timedf = prediction.loc[(prediction['month'] == datetime.datetime.now().month) 
                            & (prediction['day']== datetime.datetime.now().day) 
                            & (prediction['hour'] == datetime.datetime.now().hour)].reset_index()
    timedf.drop(['index','_id'], inplace=True, axis=1)
    timedf = timedf.set_index('user')
    # remove duplicate index
    timedf = timedf.loc[~timedf.index.duplicated(keep='last')]
    for i in timedf.index:
        timedf[i] = 0
    for index, row in timedf.iterrows():
        for i in timedf.index:
            timedf.loc[index,i] = timedf.loc[i,'totalsteps'] - timedf.loc[index,'totalsteps']
    timedf.drop(['prediction', 'totalsteps', 'type'], inplace=True, axis=1)
    timedf.reset_index(inplace=True)
#     print(timedf)
    for index, rows in timedf.iterrows():
#         print(rows[0])
        for i in range(5,len(rows)):
#             print(timedf.columns[i],rows[i])
            if (int(rows[i]) <= 50) and (int(rows[i]) >= 20):
                mqttc = mqtt.Client()
                mqttc.connect(server, port)
                mqttc.publish(rows[0], "快被超越" + rows[0] + "差" + str(int(rows[i])) +"步")
            elif (int(rows[i]) >= -50) and (int(rows[i]) <= -20):
                mqttc = mqtt.Client()
                mqttc.connect(server, port)
                mqttc.publish(rows[0], "快超越" + rows[0] + "差" + str(int(rows[i])) +"步")
    records = timedf.to_dict('records')
    if records != []:
        collection = db.rank
        collection.delete_many({})
        collection.insert_many(records)
    else:
        print('records', records)
    conn.close()

In [67]:
insert_rank_relationship()